In [33]:
import pandas as pd
import numpy as np
import math

In [34]:
stations = pd.read_csv('../data/modelInput/stations_201505_201611.csv')
station_ids = stations['station_id'].unique()

In [35]:
trips = pd.read_csv('../data/raw/201505-201611-hubway-tripdata.csv')

In [36]:
trips.columns

Index(['starttime', 'stoptime', 'start_station_id', 'start_station_name',
       'start_station_latitude', 'start_station_longitude', 'end_station_id',
       'end_station_name', 'end_station_latitude', 'end_station_longitude'],
      dtype='object')

In [37]:
#round start/end timestamps of trips to the full hour
def set_date_hour(timestamp_string):
    timestamp = pd.Timestamp(timestamp_string)
    return pd.Timestamp(timestamp.year, timestamp.month, timestamp.day, timestamp.hour)

trips['start_date_hour'] = trips['starttime'].apply(set_date_hour)
trips['end_date_hour'] = trips['stoptime'].apply(set_date_hour)

In [38]:
#split trips into start and end data
#add new features, counting arrivals and departures 

start_trips = trips[['start_date_hour', 'start_station_id']]
start_trips = start_trips.rename(columns={'start_date_hour': 'date_hour', 'start_station_id': 'station_id'})
start_trips = start_trips[start_trips['station_id'].isin(station_ids)]
start_trips['arrivals'] = 0
start_trips['departures'] = 1
start_trips.head()

,date_hour,station_id,arrivals,departures
0,2015-05-01,115,0,1
1,2015-05-01,74,0,1
2,2015-05-01,74,0,1
3,2015-05-01,36,0,1
4,2015-05-01,9,0,1


In [39]:
end_trips = trips[['end_date_hour', 'end_station_id']]
end_trips = end_trips.rename(columns={'end_date_hour': 'date_hour', 'end_station_id': 'station_id'})
end_trips = end_trips[end_trips['station_id'].isin(station_ids)]
end_trips['arrivals'] = 1
end_trips['departures'] = 0
end_trips.head()

,date_hour,station_id,arrivals,departures
0,2015-05-01 00:00:00,74,1,0
1,2015-05-01 00:00:00,104,1,0
2,2015-05-01 00:00:00,75,1,0
3,2015-05-01 00:00:00,45,1,0
4,2015-05-01 02:00:00,10,1,0


In [40]:
#merge both sets into one
trips = start_trips.append(end_trips)

In [41]:
#fill all missing hours in the trip data with dummy entries

#get all unique sations, date/hour combinations and create a multi-index from the cross-product
date_hours = pd.date_range(pd.Timestamp('2015-05-01 00:00:00'), pd.Timestamp('2016-11-30 23:00:00'), freq='H')
date_hours_station_index = pd.MultiIndex.from_product([date_hours, station_ids], names=['date_hour', 'station_id'])

#fill arrivals and departues with zeros
hour_dummies = pd.DataFrame({'arrivals': 0, 'departures': 0}, index=date_hours_station_index)
hour_dummies = hour_dummies.reset_index()
hour_dummies.head()

,date_hour,station_id,arrivals,departures
0,2015-05-01,3,0,0
1,2015-05-01,4,0,0
2,2015-05-01,5,0,0
3,2015-05-01,6,0,0
4,2015-05-01,7,0,0


In [42]:
#append dummy values
trips = trips.append(hour_dummies)

#aggregate by hour and station id
trips = trips.groupby(['date_hour', 'station_id']).sum()

#the difference of arrivals and departures is the actual flow at a station
trips['flow'] = trips['arrivals'] - trips['departures']

#localize date/hour to EST, so that merging with weather data is easy
trips = trips.reset_index()
trips = trips.set_index('date_hour')
trips = trips.tz_localize('EST')
trips.head()

,station_id,arrivals,departures,flow
date_hour,,,,
2015-05-01 00:00:00-05:00,3,0,0,0
2015-05-01 00:00:00-05:00,4,0,0,0
2015-05-01 00:00:00-05:00,5,0,0,0
2015-05-01 00:00:00-05:00,6,0,0,0
2015-05-01 00:00:00-05:00,7,0,0,0


In [43]:
trips.to_csv('../data/modelInput/flowPerHourAndStation.csv')